# 📥 FASE 1: Extracción Automática (WOL)

In [ ]:
# =============================================================================
#  🤖 ASISTENTE DE PROGRAMACIÓN VMC - FASE 1: EXTRACCIÓN DE DATOS (WOL)
# =============================================================================
#  DESCRIPCIÓN GENERAL:
#  Este programa se conecta a la Biblioteca en Línea (WOL), descarga la info
#  de las reuniones del mes seleccionado y la guarda en tu hoja 'Info-reunion'.
#  Funciona de manera interactiva pidiendo Año y Mes.
# =============================================================================
#  🛠️ ÚLTIMAS CORRECCIONES APLICADAS (ENERO 2026):
#  1. Limpieza automática: Borra datos viejos de 'Info-reunion' antes de escribir
#     para evitar duplicados al final de la hoja.
#  2. Mapeo estricto: Se fuerza el orden de las columnas para evitar que la
#     'Lectura de la Biblia' aparezca en 'Maestros Título 1'.
# =============================================================================

import requests
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import re
import calendar
from datetime import datetime, timedelta
import time
import os

# Detectar entorno para Google Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

# ==========================================
# 1. CONFIGURACIÓN
# ==========================================

RUTA_ARCHIVO = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/Programación VMC_Septiembre-2025-2026.xlsx'

# DEFINIMOS EL ORDEN EXACTO DE LAS COLUMNAS (Para solucionar el desorden en excel)
COLUMNAS_ORDENADAS = [
    'Semana',
    'Libro',
    'Canción Inicial',
    'Tesoros de la Biblia',
    'Segunda Canción',
    'Tercera Canción',
    'Maestros Título 1',
    'Maestros Título 2',
    'Maestros Título 3',
    'Maestros Título 4',
    'NVC Título 1',
    'NVC Título 2',
    'NVC Título 3',
    'Info Lectura Biblia',
    'Info Estudio Libro'
]

def obtener_fecha_interactiva():
    print("📅 CONFIGURACIÓN DE BÚSQUEDA")
    print("----------------------------")
    try:
        anio = int(input("👉 Ingresa el AÑO (ej. 2026): "))
        mes = int(input("👉 Ingresa el MES (1-12): "))
        return anio, mes
    except ValueError:
        return None, None

# ==========================================
# 2. GENERADOR DE URLs
# ==========================================

def generar_urls_dinamicas(anio, mes):
    print(f"\n🔍 Calculando semanas para: {mes}/{anio}...")
    urls = []
    c = calendar.Calendar(firstweekday=calendar.MONDAY)
    monthcal = c.monthdatescalendar(anio, mes)

    for week in monthcal:
        monday = week[0]
        # Si el lunes cae en el mes, generamos la URL
        if monday.month == mes:
            # Usamos el enlace del texto diario que redirige a la reunión
            url = f"https://wol.jw.org/es/wol/dt/r4/lp-s/{monday.year}/{monday.month:02d}/{monday.day:02d}"
            if url not in urls:
                urls.append(url)
    return urls

# ==========================================
# 3. EXTRACCIÓN (NÚCLEO)
# ==========================================

def limpiar_texto_estudio(texto):
    if not texto: return ""
    libros = ['lfb', 'bt', 'lmd', 'lvs', 'cf', 'rr', 'ia', 'jr']
    texto_limpio = texto
    for libro in libros:
        patron = rf"\b{libro}(?=[a-zA-Z0-9])"
        texto_limpio = re.sub(patron, f"{libro} ", texto_limpio, flags=re.IGNORECASE)
    return texto_limpio

def extraer_informacion(url):
    HEADERS = {'User-Agent': 'Mozilla/5.0'}
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Inicializamos el diccionario con las claves en orden correcto y vacías
        info = {col: "" for col in COLUMNAS_ORDENADAS}

        # Variables temporales para listas
        titulos_maestros = []
        nvc_titulos = []

        # --- A. Datos Básicos ---
        semana_elem = soup.find('h1', id='p1')
        info['Semana'] = semana_elem.get_text(strip=True) if semana_elem else "Semana desconocida"

        libro_elem = soup.find('h2', id='p2')
        if libro_elem:
            strongs = libro_elem.find_all('strong')
            info['Libro'] = " ".join([s.get_text(strip=True) for s in strongs]) if strongs else libro_elem.get_text(strip=True)

        # --- B. Tesoros ---
        posibles = soup.find_all(lambda tag: tag.name in ['h3', 'p'] and "(10 mins.)" in tag.text)
        for elem in posibles:
            if "Perlas escondidas" in elem.text: continue
            clean = elem.get_text(strip=True).replace("(10 mins.)", "").strip()
            if clean:
                info['Tesoros de la Biblia'] = clean
                break

        if not info['Tesoros de la Biblia']:
            elem_p5 = soup.find(id='p5')
            if elem_p5: info['Tesoros de la Biblia'] = elem_p5.get_text(strip=True).replace("(10 mins.)", "").strip()

        # --- C. Canciones ---
        canciones = []
        for h3 in soup.find_all('h3'):
            txt = h3.get_text(strip=True)
            if "Canción" in txt:
                m = re.search(r'Canción\s+(\d+)', txt)
                if m: canciones.append(f"Canción {m.group(1)}")

        if len(canciones) > 0: info['Canción Inicial'] = canciones[0]
        if len(canciones) > 1: info['Segunda Canción'] = canciones[1]
        if len(canciones) > 2: info['Tercera Canción'] = canciones[2]

        # --- D. Lectura y Estudio ---
        parrafos = soup.find_all('p')
        for p in parrafos:
            txt = p.get_text(strip=True)
            # Lectura (Busca "4 mins" o similar)
            if "(4 mins.)" in txt and not info['Info Lectura Biblia']:
                 m = re.search(r'\((?:4|3)\s+mins?.*?\)\s*(.+)', txt)
                 if m: info['Info Lectura Biblia'] = m.group(1).strip()
            # Estudio (Busca "30 mins")
            if "(30 mins.)" in txt and not info['Info Estudio Libro']:
                m = re.search(r'\(30\s+mins?.*?\)\s*(.+)', txt)
                if m: info['Info Estudio Libro'] = limpiar_texto_estudio(m.group(1).strip())

        # --- E. Maestros y Vida (Escaneo Secuencial) ---
        seccion = None
        headers = soup.find_all(['h2', 'h3'])

        for h in headers:
            txt = h.get_text(strip=True)

            if h.name == 'h2':
                if "SEAMOS MEJORES MAESTROS" in txt.upper(): seccion = "MAESTROS"
                elif "NUESTRA VIDA CRISTIANA" in txt.upper(): seccion = "VIDA"
                elif "TESOROS" not in txt.upper(): pass

            elif h.name == 'h3' and seccion:
                if "Canción" in txt or "Palabras de conclusión" in txt: continue
                if "Artículo de estudio" in txt: break

                if seccion == "MAESTROS":
                    titulos_maestros.append(txt)
                elif seccion == "VIDA":
                    # Si detectamos Estudio Bíblico o Conclusión, no lo agregamos como título NVC normal
                    if "Estudio bíblico de la congregación" in txt:
                         pass
                    else:
                        nvc_titulos.append(txt)

        # Asignar Maestros a columnas específicas
        for i, titulo in enumerate(titulos_maestros):
            if i < 4: info[f'Maestros Título {i+1}'] = titulo

        # Asignar NVC
        if len(nvc_titulos) > 0: info['NVC Título 1'] = nvc_titulos[0]
        if len(nvc_titulos) > 1: info['NVC Título 2'] = nvc_titulos[1]

        # El NVC 3 siempre suele ser el Estudio
        info['NVC Título 3'] = "Estudio bíblico de la congregación"

        return info

    except Exception as e:
        print(f"❌ Error leyendo {url}: {e}")
        return None

# ==========================================
# 4. GUARDADO (LIMPIEZA + ESCRITURA)
# ==========================================

def guardar_excel_limpio(datos, ruta):
    print(f"\n💾 Actualizando archivo: {os.path.basename(ruta)}...")
    try:
        book = load_workbook(ruta)
        if 'Info-reunion' not in book.sheetnames:
            book.create_sheet('Info-reunion')

        ws = book['Info-reunion']

        # 1. LIMPIEZA: Borrar contenido desde la fila 2 hacia abajo
        num_filas = ws.max_row
        if num_filas > 1:
            print(f"   🧹 Limpiando {num_filas-1} filas antiguas...")
            ws.delete_rows(2, amount=num_filas-1)

        # 2. ESCRITURA: Escribir los nuevos datos
        print(f"   ✍️ Escribiendo {len(datos)} semanas nuevas...")

        # Convertimos la lista de diccionarios a DataFrame asegurando el orden
        df_new = pd.DataFrame(datos, columns=COLUMNAS_ORDENADAS)

        rows = dataframe_to_rows(df_new, index=False, header=False)
        for r_idx, row in enumerate(rows, 1):
            for c_idx, value in enumerate(row, 1):
                # Escribimos en fila r_idx + 1 (respetando encabezado)
                ws.cell(row=r_idx + 1, column=c_idx, value=value)

        book.save(ruta)
        book.close()
        print("✅ Archivo actualizado correctamente con datos limpios y ordenados.")

    except Exception as e:
        print(f"❌ Error Excel: {e}")

# ==========================================
# 5. EJECUCIÓN
# ==========================================
if __name__ == "__main__":
    anio, mes = obtener_fecha_interactiva()

    if anio and mes:
        urls = generar_urls_dinamicas(anio, mes)
        datos_totales = []

        print("\n🚀 INICIANDO EXTRACCIÓN...")
        for i, url in enumerate(urls, 1):
            print(f"   ({i}/{len(urls)}) Leyendo...", end=" ")
            data = extraer_informacion(url)
            if data:
                print(f"✅ {data['Semana']}")
                datos_totales.append(data)
            else:
                print("❌ Falló")
            time.sleep(1)

        if datos_totales:
            guardar_excel_limpio(datos_totales, RUTA_ARCHIVO)
            print("\n🎉 FASE 1 COMPLETADA.")
        else:
            print("\n⚠️ No se encontraron datos.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📅 CONFIGURACIÓN DE BÚSQUEDA
----------------------------
👉 Ingresa el AÑO (ej. 2026): 2026
👉 Ingresa el MES (1-12): 1

🔍 Calculando semanas para: 1/2026...

🚀 INICIANDO EXTRACCIÓN...
   (1/4) Leyendo... ✅ 5-11 DE ENERO
   (2/4) Leyendo... ✅ 12-18 DE ENERO
   (3/4) Leyendo... ✅ 19-25 DE ENERO
   (4/4) Leyendo... ✅ 26 DE ENERO A 1 DE FEBRERO

💾 Actualizando archivo: Programación VMC_Septiembre-2025-2026.xlsx...
   🧹 Limpiando 993 filas antiguas...
   ✍️ Escribiendo 4 semanas nuevas...
✅ Archivo actualizado correctamente con datos limpios y ordenados.

🎉 FASE 1 COMPLETADA.


# 🚀 FASE 2: Fusión y Generación del Formato

In [ ]:
# =============================================================================
#  🤖 ASISTENTE DE PROGRAMACIÓN VMC - FASE 1: EXTRACCIÓN HÍBRIDA (Excel + JSON)
# =============================================================================
#  RAMA: fix-fase1
#  CAMBIOS:
#  1. Genera Excel (Legacy) para respaldo.
#  2. Genera JSON (Nuevo) para el motor de asignación automática.
#  3. Soporta Enero 2026 correctamente.
# =============================================================================

import requests
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import re
import calendar
import json
import time
import os

# Detectar entorno
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

# CONFIGURACIÓN
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
RUTA_EXCEL = os.path.join(RUTA_BASE, 'Programación VMC_Septiembre-2025-2026.xlsx')
RUTA_JSON = os.path.join(RUTA_BASE, 'data_reuniones_2026.json')

COLUMNAS_ORDENADAS = [
    'Semana', 'Libro', 'Canción Inicial', 'Tesoros de la Biblia',
    'Segunda Canción', 'Tercera Canción',
    'Maestros Título 1', 'Maestros Título 2', 'Maestros Título 3', 'Maestros Título 4',
    'NVC Título 1', 'NVC Título 2', 'NVC Título 3',
    'Info Lectura Biblia', 'Info Estudio Libro'
]

def obtener_fecha_interactiva():
    print("📅 CONFIGURACIÓN DE BÚSQUEDA")
    try:
        anio = int(input("👉 AÑO (ej. 2026): "))
        mes = int(input("👉 MES (1-12): "))
        return anio, mes
    except: return None, None

def generar_urls_dinamicas(anio, mes):
    print(f"\n🔍 Calculando semanas para: {mes}/{anio}...")
    urls = []
    c = calendar.Calendar(firstweekday=calendar.MONDAY)
    for week in c.monthdatescalendar(anio, mes):
        if week[0].month == mes:
            urls.append(f"https://wol.jw.org/es/wol/dt/r4/lp-s/{week[0].year}/{week[0].month:02d}/{week[0].day:02d}")
    return urls

def limpiar_texto_estudio(texto):
    if not texto: return ""
    libros = ['lfb', 'bt', 'lmd', 'lvs', 'cf', 'rr', 'ia', 'jr']
    for l in libros: texto = re.sub(rf"\b{l}(?=[a-zA-Z0-9])", f"{l} ", texto, flags=re.IGNORECASE)
    return texto

def extraer_informacion(url):
    try:
        soup = BeautifulSoup(requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).content, 'html.parser')
        info = {col: "" for col in COLUMNAS_ORDENADAS}

        # Básicos
        info['Semana'] = soup.find('h1', id='p1').get_text(strip=True) if soup.find('h1', id='p1') else "N/A"
        if soup.find('h2', id='p2'): info['Libro'] = soup.find('h2', id='p2').get_text(strip=True)

        # Tesoros (Lógica Refinada S-38: Discurso de 10 mins)
        for elem in soup.find_all(lambda t: t.name in ['h3','p'] and "(10 mins.)" in t.text):
            if "Perlas" not in elem.text:
                info['Tesoros de la Biblia'] = elem.get_text(strip=True).replace("(10 mins.)", "").strip()
                break

        # Canciones
        canciones = [f"Canción {re.search(r'(\d+)', h3.text).group(1)}" for h3 in soup.find_all('h3') if "Canción" in h3.text and re.search(r'(\d+)', h3.text)]
        if len(canciones) > 0: info['Canción Inicial'] = canciones[0]
        if len(canciones) > 1: info['Segunda Canción'] = canciones[1]
        if len(canciones) > 2: info['Tercera Canción'] = canciones[2]

        # Lectura y Estudio
        for p in soup.find_all('p'):
            if "(4 mins.)" in p.text and not info['Info Lectura Biblia']:
                info['Info Lectura Biblia'] = re.search(r'\((?:4|3)\s+mins?.*?\)\s*(.+)', p.text).group(1).strip()
            if "(30 mins.)" in p.text and not info['Info Estudio Libro']:
                info['Info Estudio Libro'] = limpiar_texto_estudio(re.search(r'\(30\s+mins?.*?\)\s*(.+)', p.text).group(1).strip())

        # Secciones Maestros y Vida
        seccion, titulos_maestros, nvc_titulos = None, [], []
        for h in soup.find_all(['h2', 'h3']):
            txt = h.get_text(strip=True)
            if h.name == 'h2':
                if "MEJORES MAESTROS" in txt.upper(): seccion = "MAESTROS"
                elif "VIDA CRISTIANA" in txt.upper(): seccion = "VIDA"
            elif h.name == 'h3' and seccion:
                if any(x in txt for x in ["Canción", "Conclusión", "Artículo"]): continue
                if "Artículo de estudio" in txt: break

                if seccion == "MAESTROS": titulos_maestros.append(txt)
                elif seccion == "VIDA" and "Estudio bíblico" not in txt: nvc_titulos.append(txt)

        for i, t in enumerate(titulos_maestros[:4]): info[f'Maestros Título {i+1}'] = t
        for i, t in enumerate(nvc_titulos[:3]): info[f'NVC Título {i+1}'] = t
        info['NVC Título 3'] = "Estudio bíblico de la congregación" # Default

        return info
    except Exception as e:
        print(f"❌ Error {url}: {e}")
        return None

def guardar_excel_limpio(datos):
    print(f"\n💾 (LEGACY) Guardando Excel: {os.path.basename(RUTA_EXCEL)}...")
    try:
        book = load_workbook(RUTA_EXCEL) if os.path.exists(RUTA_EXCEL) else pd.ExcelWriter(RUTA_EXCEL, engine='openpyxl').book
        if 'Info-reunion' not in book.sheetnames: book.create_sheet('Info-reunion')
        ws = book['Info-reunion']

        if ws.max_row > 1: ws.delete_rows(2, amount=ws.max_row-1) # Limpiar

        for r_idx, row in enumerate(dataframe_to_rows(pd.DataFrame(datos, columns=COLUMNAS_ORDENADAS), index=False, header=False), 1):
            for c_idx, val in enumerate(row, 1): ws.cell(row=r_idx+1, column=c_idx, value=val)

        book.save(RUTA_EXCEL)
        print("✅ Excel actualizado.")
    except Exception as e: print(f"❌ Error Excel: {e}")

# --- AQUÍ ESTÁ LA NUEVA FUNCIÓN JSON ---
def guardar_json_reuniones(datos):
    print(f"\n💾 (NUEVO) Generando JSON para Motor de Asignación...")
    data_estructurada = []

    for d in datos:
        # Aquí aplicamos lógica básica basada en S-38 para definir requisitos
        semana_obj = {
            "semana": d['Semana'],
            "partes": [
                {"tipo": "Presidente", "titulo": "Presidencia", "requiere": "ANC", "tiempo": 0},
                {"tipo": "Oracion_Inicio", "titulo": "Oración", "requiere": "H_BAUTIZADO", "tiempo": 5},
                {"tipo": "Tesoros", "titulo": d['Tesoros de la Biblia'], "requiere": "ANC_SM", "tiempo": 10},
                {"tipo": "Perlas", "titulo": "Perlas Escondidas", "requiere": "ANC_SM", "tiempo": 10},
                {"tipo": "Lectura", "titulo": d['Info Lectura Biblia'], "requiere": "EST_VARON", "tiempo": 4}
            ]
        }

        # Maestros (Dinámico)
        for i in range(1, 5):
            titulo = d.get(f'Maestros Título {i}')
            if titulo:
                # Análisis simple de texto para inferir si es Discurso (Solo varones)
                req = "ESTUDIANTE"
                if "Discurso" in titulo: req = "EST_VARON"
                semana_obj['partes'].append({"tipo": "Maestros", "titulo": titulo, "requiere": req, "tiempo": 5})

        # Vida Cristiana
        for i in range(1, 3):
            titulo = d.get(f'NVC Título {i}')
            if titulo:
                semana_obj['partes'].append({"tipo": "NVC", "titulo": titulo, "requiere": "ANC_SM", "tiempo": 15})

        semana_obj['partes'].extend([
            {"tipo": "Estudio_Libro", "titulo": "Estudio Bíblico", "requiere": "ANC_SM", "tiempo": 30},
            {"tipo": "Lector_Libro", "titulo": "Lectura EBC", "requiere": "H_BAUTIZADO", "tiempo": 0},
            {"tipo": "Oracion_Final", "titulo": "Oración", "requiere": "H_BAUTIZADO", "tiempo": 5}
        ])

        data_estructurada.append(semana_obj)

    with open(RUTA_JSON, 'w', encoding='utf-8') as f:
        json.dump(data_estructurada, f, indent=4, ensure_ascii=False)
    print(f"✅ JSON guardado en: {os.path.basename(RUTA_JSON)}")

# EJECUCIÓN
if __name__ == "__main__":
    anio, mes = obtener_fecha_interactiva()
    if anio:
        datos = [extraer_informacion(url) for url in generar_urls_dinamicas(anio, mes) if extraer_informacion(url)]
        if datos:
            guardar_excel_limpio(datos)      # Mantenemos el puente viejo
            guardar_json_reuniones(datos)    # Construimos el puente nuevo
            print("\n🎉 FASE 1 COMPLETADA (HÍBRIDA).")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📅 CONFIGURACIÓN DE BÚSQUEDA
👉 AÑO (ej. 2026): 2026
👉 MES (1-12): 1

🔍 Calculando semanas para: 1/2026...

💾 (LEGACY) Guardando Excel: Programación VMC_Septiembre-2025-2026.xlsx...
✅ Excel actualizado.

💾 (NUEVO) Generando JSON para Motor de Asignación...
✅ JSON guardado en: data_reuniones_2026.json

🎉 FASE 1 COMPLETADA (HÍBRIDA).


# Utilidad: Covertir Students.csv en Base de datos


In [ ]:
# =============================================================================
#  🔄 MIGRACIÓN DE DATOS: DE CLM EXPLORER A VMC PYTHON
# =============================================================================
#  Este script toma tu archivo 'Students.csv' y lo convierte en una base de datos
#  limpia y estructurada con banderas (flags) para cada asignación.
# =============================================================================

import pandas as pd
import os

# CONFIGURACIÓN
RUTA_ENTRADA = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/Students.csv'
RUTA_SALIDA = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/base_datos_hermanos.csv'

def procesar_base_datos():
    if not os.path.exists(RUTA_ENTRADA):
        print(f"❌ No encuentro el archivo: {RUTA_ENTRADA}")
        return

    print("📖 Leyendo archivo original...")
    try:
        # Leemos el CSV. A veces CLM Explorer usa codificación diferente, probamos 'utf-8' o 'latin-1'
        df_orig = pd.read_csv(RUTA_ENTRADA, encoding='utf-8')
    except:
        df_orig = pd.read_csv(RUTA_ENTRADA, encoding='latin-1')

    # Lista para guardar los datos procesados
    datos_nuevos = []

    print("⚙️ Procesando hermanos...")

    for _, row in df_orig.iterrows():
        # 1. DATOS BÁSICOS
        es_hermano = True if str(row['Gender']).strip() == 'Hermano' else False

        # Mapeo de Privilegios
        rol_ingles = str(row['Role']).strip()
        privilegio = 'PUB'
        if rol_ingles == 'Elder': privilegio = 'ANC'
        elif rol_ingles == 'MS': privilegio = 'SM'

        # Limpieza de "Use For" (Capacidades)
        usos = str(row['Use For']).lower() if pd.notna(row['Use For']) else ""

        # 2. DEFINICIÓN DE BANDERAS (FLAGS)
        # Aquí traducimos los códigos de CLM Explorer a Columnas Booleanas
        flags = {
            'id': row['Student ID'],
            'nombre_completo': row['Alt Name'],
            'genero': 'M' if es_hermano else 'F',
            'privilegio': privilegio,
            'activo': 1 if str(row['Active']) == 'Y' else 0,

            # Asignaciones de Plataforma
            'f_presidente': 1 if 'presi' in usos else 0,
            'f_oracion': 1 if 'orac' in usos else 0,
            'f_lectura': 1 if 'lectura' in usos else 0,
            'f_tesoros': 1 if 'teso' in usos else 0, # A.Teso
            'f_perlas': 1 if 'perlas' in usos else 0,
            'f_vida': 1 if 'vida' in usos else 0,
            'f_estudio_libro': 1 if 'cbs' in usos else 0, # CBS = Congregation Bible Study
            'f_lector_libro': 1 if 'lector ebc' in usos else 0,

            # Asignaciones Estudiantiles
            'f_discurso_est': 1 if 'discurso' in usos else 0,
            'f_demos': 1 if 'demos' in usos else 0, # Incluye Ayudante normalmente
            'f_ayudante': 1 if 'asis' in usos or 'demos' in usos else 0,

            'observaciones': row['Remarks'] if pd.notna(row['Remarks']) else ""
        }

        datos_nuevos.append(flags)

    # 3. CREAR DATAFRAME Y GUARDAR
    df_nuevo = pd.DataFrame(datos_nuevos)

    # Reordenar columnas para que se vea ordenado
    cols_orden = ['id', 'nombre_completo', 'genero', 'privilegio', 'activo',
                  'f_presidente', 'f_oracion', 'f_lectura', 'f_tesoros', 'f_perlas',
                  'f_vida', 'f_estudio_libro', 'f_lector_libro',
                  'f_discurso_est', 'f_demos', 'f_ayudante', 'observaciones']

    df_nuevo = df_nuevo[cols_orden]

    df_nuevo.to_csv(RUTA_SALIDA, index=False, encoding='utf-8-sig') # utf-8-sig para que Excel lo abra bien con tildes

    print(f"✅ ¡Éxito! Base de datos convertida.")
    print(f"📂 Archivo guardado en: {os.path.basename(RUTA_SALIDA)}")
    print(f"📊 Total procesados: {len(df_nuevo)} hermanos")

    # Mostrar muestra
    print("\n🔍 Vista previa:")
    print(df_nuevo[['nombre_completo', 'privilegio', 'f_presidente', 'f_lectura']].head())

# Ejecutar
if __name__ == "__main__":
    procesar_base_datos()

📖 Leyendo archivo original...
⚙️ Procesando hermanos...
✅ ¡Éxito! Base de datos convertida.
📂 Archivo guardado en: base_datos_hermanos.csv
📊 Total procesados: 101 hermanos

🔍 Vista previa:
      nombre_completo privilegio  f_presidente  f_lectura
0    Carolina Aguilar        PUB             0          0
1          Gilma Alba        PUB             0          0
2          Jorge Alba         SM             0          0
3      Eliana Alfonso        PUB             0          0
4  Juan Pablo Alfonso        ANC             1          0


# Uilidad 2: Agregar hermano nuevo

In [ ]:
# =============================================================================
#  🛠️ GESTIÓN DE HERMANOS: AGREGAR NUEVO (SIN ROMPER IDs)
# =============================================================================

import pandas as pd
import os

RUTA_BD = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/base_datos_hermanos.csv'

def agregar_hermano_manual():
    print("👤 INSCRIPCIÓN DE NUEVO HERMANO")
    print("-------------------------------")

    # 1. Cargar base de datos actual
    if not os.path.exists(RUTA_BD):
        print("❌ Error: No existe la base de datos.")
        return

    df = pd.read_csv(RUTA_BD)

    # 2. Calcular ID Automático (Max ID + 1) para integridad
    # Si la tabla está vacía, empieza en 1. Si no, busca el máximo.
    nuevo_id = 1
    if not df.empty:
        nuevo_id = df['id'].max() + 1

    print(f"🆔 ID Asignado Automáticamente: {nuevo_id}")

    # 3. Pedir datos
    nombre = input("Nombre Completo: ").strip().title()
    genero = input("Género (M/F): ").strip().upper()
    privilegio = input("Privilegio (PUB/SM/ANC): ").strip().upper()

    # Capacidades básicas (Preguntas rápidas)
    print("\n--- Capacidades (S/N) ---")
    lectura = 1 if input("¿Hace Lectura de Biblia? ").upper() == 'S' else 0
    demos = 1 if input("¿Hace Demostraciones/Ayudante? ").upper() == 'S' else 0
    discurso = 1 if input("¿Hace Discursos Estudiantiles? ").upper() == 'S' else 0

    # 4. Crear el diccionario con la estructura EXACTA
    nuevo_hermano = {
        'id': nuevo_id,
        'nombre_completo': nombre,
        'genero': genero,
        'privilegio': privilegio,
        'activo': 1,
        # Banderas
        'f_presidente': 0, 'f_oracion': 0,
        'f_lectura': lectura,
        'f_tesoros': 0, 'f_perlas': 0, 'f_vida': 0,
        'f_estudio_libro': 0, 'f_lector_libro': 0,
        'f_discurso_est': discurso,
        'f_demos': demos,
        'f_ayudante': demos, # Generalmente si hace demos, es ayudante
        'observaciones': "Ingreso manual"
    }

    # 5. Guardar
    # Convertimos el diccionario a DataFrame y lo concatenamos
    nuevo_df = pd.DataFrame([nuevo_hermano])

    # Alineamos columnas (para evitar desorden si faltan claves)
    # Usamos las columnas del df original como molde
    for col in df.columns:
        if col not in nuevo_df.columns:
            nuevo_df[col] = 0 # Rellenar con 0 lo que no preguntamos

    df_final = pd.concat([df, nuevo_df], ignore_index=True)
    df_final.to_csv(RUTA_BD, index=False, encoding='utf-8-sig')

    print(f"\n✅ ¡Listo! {nombre} ha sido inscrito con el ID {nuevo_id}.")

# Para usarlo, simplemente ejecuta:
# agregar_hermano_manual()

# Previo: Inicializar la Memoria (Historial Asginaciones)

In [2]:
# =============================================================================
#  🛠️ UTILIDAD: INICIALIZAR HISTORIAL (CON AUTO-CONEXIÓN DRIVE)
# =============================================================================

import pandas as pd
import os
from google.colab import drive # Importamos la librería de Drive

def inicializar_historial():
    # --- PASO 1: VERIFICAR CONEXIÓN A DRIVE ---
    if not os.path.exists('/content/drive'):
        print("🔌 Conectando a Google Drive...")
        drive.mount('/content/drive')
    else:
        print("✅ Google Drive ya está conectado.")

    # --- PASO 2: CONFIGURACIÓN DE RUTA ---
    # Verifica que esta ruta sea EXACTAMENTE la de tu carpeta
    RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
    RUTA_HISTORIAL = os.path.join(RUTA_BASE, 'historial_asignaciones.csv')

    # Verificar si la carpeta existe antes de intentar guardar
    if not os.path.exists(RUTA_BASE):
        print(f"❌ ERROR CRÍTICO: No encuentro la carpeta en Drive.")
        print(f"   Buscaba: {RUTA_BASE}")
        print("   👉 Verifica si el nombre de la carpeta tiene espacios o tildes diferentes.")
        return

    print(f"\n📂 Verificando historial en: {os.path.basename(RUTA_HISTORIAL)}")

    if os.path.exists(RUTA_HISTORIAL):
        print(f"⚠️ El archivo ya existe. No se ha borrado nada por seguridad.")
    else:
        columnas_maestras = [
            'fecha_asignacion', 'semana_texto', 'id_hermano',
            'nombre_hermano', 'tipo_asignacion', 'sala'
        ]

        df_vacio = pd.DataFrame(columns=columnas_maestras)
        df_vacio.to_csv(RUTA_HISTORIAL, index=False, encoding='utf-8')
        print(f"✅ ¡ÉXITO! Archivo de historial creado y listo.")

if __name__ == "__main__":
    inicializar_historial()

🔌 Conectando a Google Drive...
Mounted at /content/drive

📂 Verificando historial en: historial_asignaciones.csv
✅ ¡ÉXITO! Archivo de historial creado y listo.


# 🟩 Código Fase C: El Motor de Asignación v1.0

In [4]:
# =============================================================================
#  🧠 MOTOR DE ASIGNACIÓN VMC - FASE C (V2.0 SELECTIVO)
# =============================================================================
#  NUEVO: Permite elegir si asignar TODO, SOLO ESTUDIANTES o SOLO NO ESTUDIANTES.
#  Ideal si ya asignaste manualmente una sección y no quieres sobrescribirla.
# =============================================================================

import pandas as pd
import json
import os
from datetime import datetime

# RUTAS
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
FILE_DB_HERMANOS = os.path.join(RUTA_BASE, 'base_datos_hermanos.csv')
FILE_HISTORIAL = os.path.join(RUTA_BASE, 'historial_asignaciones.csv')
FILE_JSON_INPUT = os.path.join(RUTA_BASE, 'data_reuniones_2026.json')
FILE_JSON_OUTPUT = os.path.join(RUTA_BASE, 'programa_asignado_2026.json')

# CATEGORÍAS DE PARTES
PARTES_ESTUDIANTILES = ['Lectura', 'Maestros']
PARTES_NO_ESTUDIANTILES = [
    'Presidente', 'Oracion_Inicio', 'Oracion_Final',
    'Tesoros', 'Perlas', 'NVC',
    'Estudio_Libro', 'Lector_Libro'
]

REGLAS_ASIGNACION = {
    "Presidente": "f_presidente",
    "Oracion_Inicio": "f_oracion",
    "Oracion_Final": "f_oracion",
    "Tesoros": "f_tesoros",
    "Perlas": "f_perlas",
    "Lectura": "f_lectura",
    "Maestros": "f_demos",
    "NVC": "f_vida",
    "Estudio_Libro": "f_estudio_libro",
    "Lector_Libro": "f_lector_libro"
}

def cargar_datos():
    if not os.path.exists(FILE_DB_HERMANOS) or not os.path.exists(FILE_JSON_INPUT):
        print("❌ Error: Faltan archivos base.")
        return None, None, None

    df_hermanos = pd.read_csv(FILE_DB_HERMANOS)

    if os.path.exists(FILE_HISTORIAL):
        df_historial = pd.read_csv(FILE_HISTORIAL)
        df_historial['fecha_asignacion'] = pd.to_datetime(df_historial['fecha_asignacion'], errors='coerce')
    else:
        df_historial = pd.DataFrame(columns=['fecha_asignacion', 'semana_texto', 'id_hermano', 'nombre_hermano', 'tipo_asignacion', 'sala'])

    with open(FILE_JSON_INPUT, 'r', encoding='utf-8') as f:
        data_reuniones = json.load(f)

    return df_hermanos, df_historial, data_reuniones

def buscar_mejor_candidato(tipo_parte, df_hermanos, df_historial, excluidos_ids):
    columna_requisito = REGLAS_ASIGNACION.get(tipo_parte)
    if not columna_requisito: return None

    candidatos = df_hermanos[
        (df_hermanos['activo'] == 1) &
        (df_hermanos[columna_requisito] == 1)
    ].copy()

    candidatos = candidatos[~candidatos['id'].isin(excluidos_ids)]

    if candidatos.empty: return None

    scores = []
    for _, hermano in candidatos.iterrows():
        h_id = hermano['id']
        historial_especifico = df_historial[
            (df_historial['id_hermano'] == h_id) &
            (df_historial['tipo_asignacion'] == tipo_parte)
        ]

        if historial_especifico.empty:
            ultima_fecha = datetime(2000, 1, 1)
        else:
            ultima_fecha = historial_especifico['fecha_asignacion'].max()

        scores.append({'id': h_id, 'nombre': hermano['nombre_completo'], 'ultima_fecha': ultima_fecha})

    df_scores = pd.DataFrame(scores)
    # Ordenar por fecha (el que lleva más tiempo sin hacerlo va primero)
    df_scores = df_scores.sample(frac=1, random_state=42).sort_values('ultima_fecha', ascending=True)

    return df_scores.iloc[0]

def ejecutar_asignaciones_semanales():
    print("🚀 MOTOR DE ASIGNACIÓN VMC v2.0")
    print("=================================")

    # --- MENÚ SELECTOR ---
    print("\n¿Qué deseas asignar hoy?")
    print("1. TODO el programa (Completo)")
    print("2. Solo partes NO ESTUDIANTILES (Plataforma, Ancianos, NVC)")
    print("3. Solo partes ESTUDIANTILES (Seamos Mejores Maestros)")

    opcion = input("\n👉 Elige una opción (1-3): ")

    # Definir qué vamos a procesar
    tipos_a_procesar = []
    if opcion == '1':
        tipos_a_procesar = PARTES_ESTUDIANTILES + PARTES_NO_ESTUDIANTILES
        modo_texto = "PROGRAMA COMPLETO"
    elif opcion == '2':
        tipos_a_procesar = PARTES_NO_ESTUDIANTILES
        modo_texto = "SOLO NO ESTUDIANTILES"
    elif opcion == '3':
        tipos_a_procesar = PARTES_ESTUDIANTILES
        modo_texto = "SOLO ESTUDIANTILES"
    else:
        print("❌ Opción inválida.")
        return

    print(f"\n⚙️  Modo activado: {modo_texto}")

    df_hermanos, df_historial, data_reuniones = cargar_datos()
    if df_hermanos is None: return

    nuevas_asignaciones = []
    programa_final = []
    fecha_hoy = datetime.now()

    for semana in data_reuniones:
        nombre_semana = semana['semana']
        print(f"\n📅 Procesando: {nombre_semana}")

        semana_output = semana.copy()
        semana_output['asignaciones'] = []
        ocupados_esta_semana = []

        for parte in semana['partes']:
            tipo = parte['tipo']

            # --- FILTRO SELECTIVO ---
            # Si el tipo de parte NO está en la lista elegida, lo saltamos
            if tipo not in tipos_a_procesar:
                parte_asignada = parte.copy()
                parte_asignada['asignado_a'] = "** YA ASIGNADO / MANUAL **"
                semana_output['asignaciones'].append(parte_asignada)
                continue # Saltamos al siguiente ciclo

            # Si sí está, buscamos candidato
            candidato = buscar_mejor_candidato(
                tipo, df_hermanos, df_historial, ocupados_esta_semana
            )

            nombre_asignado = "PENDIENTE / SIN CANDIDATO"

            if candidato is not None:
                nombre_asignado = candidato['nombre']
                id_asignado = candidato['id']
                ocupados_esta_semana.append(id_asignado)

                nuevas_asignaciones.append({
                    'fecha_asignacion': fecha_hoy,
                    'semana_texto': nombre_semana,
                    'id_hermano': id_asignado,
                    'nombre_hermano': nombre_asignado,
                    'tipo_asignacion': tipo,
                    'sala': 'Principal'
                })

                # Actualizar memoria temporal
                nuevo_registro = pd.DataFrame([{
                    'fecha_asignacion': fecha_hoy,
                    'id_hermano': id_asignado,
                    'tipo_asignacion': tipo
                }])
                df_historial = pd.concat([df_historial, nuevo_registro], ignore_index=True)

                print(f"   ✅ {tipo:<15} -> {nombre_asignado}")
            else:
                print(f"   ⚠️ {tipo:<15} -> SIN CANDIDATO")

            parte_asignada = parte.copy()
            parte_asignada['asignado_a'] = nombre_asignado
            semana_output['asignaciones'].append(parte_asignada)

        programa_final.append(semana_output)

    # GUARDAR
    with open(FILE_JSON_OUTPUT, 'w', encoding='utf-8') as f:
        json.dump(programa_final, f, indent=4, ensure_ascii=False)

    # Guardar historial temporal solo con lo nuevo
    df_nuevas = pd.DataFrame(nuevas_asignaciones)
    if not df_nuevas.empty:
        ruta_temp = os.path.join(RUTA_BASE, 'historial_temp_revision.csv')
        df_nuevas.to_csv(ruta_temp, index=False, encoding='utf-8')
        print(f"\n💾 Programa guardado en: {os.path.basename(FILE_JSON_OUTPUT)}")
        print(f"💾 Historial nuevo en: {os.path.basename(ruta_temp)}")
        print("   (Recuerda copiar este historial al archivo maestro si apruebas los cambios)")
    else:
        print("\nℹ️  No se generaron nuevas asignaciones.")

if __name__ == "__main__":
    ejecutar_asignaciones_semanales()

🚀 MOTOR DE ASIGNACIÓN VMC v2.0

¿Qué deseas asignar hoy?
1. TODO el programa (Completo)
2. Solo partes NO ESTUDIANTILES (Plataforma, Ancianos, NVC)
3. Solo partes ESTUDIANTILES (Seamos Mejores Maestros)

👉 Elige una opción (1-3): 2

⚙️  Modo activado: SOLO NO ESTUDIANTILES

📅 Procesando: 5-11 DE ENERO
   ✅ Presidente      -> Orlando Hernandez
   ✅ Oracion_Inicio  -> Jorge Alba
   ✅ Tesoros         -> Ricardo Morales
   ✅ Perlas          -> Juan Pablo Alfonso
   ✅ NVC             -> Henry Rodriguez
   ✅ NVC             -> Cesar Hernandez
   ✅ Estudio_Libro   -> James Cabieles
   ✅ Lector_Libro    -> Guillermo Quintero
   ✅ Oracion_Final   -> Juan Carlos Riaño

📅 Procesando: 12-18 DE ENERO
   ✅ Presidente      -> Juan Pablo Alfonso
   ✅ Oracion_Inicio  -> Gustavo Ramirez
   ✅ Tesoros         -> Juan Carlos Riaño
   ✅ Perlas          -> Jorge Alba
   ✅ NVC             -> Guillermo Quintero
   ✅ Estudio_Libro   -> Cristian Calderón
   ✅ Lector_Libro    -> Cesar Hernandez
   ✅ Oracion_Final